# 07 — TensorBoard Integration (step‑by‑step)
**Goal:** add TensorBoard to any of your current lessons with minimal changes.

By the end you will:
- Install and launch TensorBoard
- Log scalars (loss/accuracy), learning rate, and model graph
- Optionally log weight histograms and a few example images


## 0) Install & launch TensorBoard
Run these in a terminal **inside your `torch` venv**:

```bash
pip install tensorboard
tensorboard --logdir runs --port 6006 --bind_all
```
Then open `http://YOUR_SERVER_IP:6006` from your browser. (`--bind_all` lets you view it from another machine on your LAN.)


## 1) Minimal example you can copy into any script
This is a tiny training loop on a synthetic dataset. Focus on the **SummaryWriter** bits — you can transplant them into your MNIST/CIFAR scripts almost verbatim.


In [ ]:
import time
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast, GradScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

# ----- Toy data (1000 samples, 32 features, 10 classes)
X = torch.randn(1000, 32)
y = torch.randint(0, 10, (1000,))
train_ds = TensorDataset(X, y)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)

# ----- Tiny model
class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(32, 64), nn.ReLU(), nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.net(x)

model = TinyNet().to(device)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
scaler = GradScaler(enabled=(device=='cuda'))

# ----- 1) Create a SummaryWriter (one per run)
run_name = f"tinynet_{int(time.time())}"
writer = SummaryWriter(log_dir=f"runs/{run_name}")
print('Logging to', f'runs/{run_name}')

# (Optional) 2) Log the computation graph once
dummy_input = torch.randn(1, 32).to(device)
writer.add_graph(model, dummy_input)

# Utility to compute accuracy
def acc(logits, y):
    return (logits.argmax(dim=1) == y).float().mean().item()

global_step = 0
EPOCHS = 5
for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss, running_acc, n = 0.0, 0.0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        with autocast(enabled=(device=='cuda')):
            logits = model(xb)
            loss = F.cross_entropy(logits, yb)
        opt.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        scaler.step(opt); scaler.update()

        # ----- 3) Log scalar metrics per training step
        step_acc = acc(logits, yb)
        writer.add_scalar('train/loss', loss.item(), global_step)
        writer.add_scalar('train/acc',  step_acc,    global_step)
        # Log current learning rate from the first param group
        writer.add_scalar('train/lr',   opt.param_groups[0]['lr'], global_step)

        bs = xb.size(0)
        running_loss += loss.item() * bs
        running_acc  += step_acc * bs
        n += bs
        global_step += 1

    # ----- 4) Log per-epoch histograms to inspect weights
    for name, p in model.named_parameters():
        writer.add_histogram(f'params/{name}', p.detach().cpu(), epoch)

    print(f"epoch {epoch}: loss {running_loss/n:.4f} acc {running_acc/n:.3f}")

writer.close()
print('Done. Open TensorBoard to see your run.')


## 2) Drop‑in snippets for your existing lessons
Below are copy‑paste chunks to integrate into **MNIST**, **CIFAR**, and the **Training Loop Anatomy** notebook.


### A) Imports
```python
from torch.utils.tensorboard import SummaryWriter
```


### B) Create the writer
Put this near your training setup (after model/optimizer are created):

```python
import time
run_name = f"mnist_cnn_{int(time.time())}"
writer = SummaryWriter(log_dir=f"runs/{run_name}")
```
Optional — log the model graph once:
```python
dummy = torch.randn(1, 1, 28, 28, device=device)  # MNIST example
writer.add_graph(model, dummy)
```


### C) Inside your **training loop** (per batch)
Add these after computing `loss` and before/after the optimizer step:

```python
global_step = 0  # define once above your loop

# ... inside the loop after you get `loss` and `logits`
train_acc = (logits.argmax(1) == yb).float().mean().item()
writer.add_scalar('train/loss', loss.item(), global_step)
writer.add_scalar('train/acc',  train_acc,   global_step)
writer.add_scalar('train/lr',   opt.param_groups[0]['lr'], global_step)
global_step += 1
```


### D) After each **epoch**
Log your validation metrics and (optionally) histograms of model weights:

```python
writer.add_scalar('val/loss', val_loss, epoch)
writer.add_scalar('val/acc',  val_acc,  epoch)

for name, p in model.named_parameters():
    writer.add_histogram(f'params/{name}', p.detach().cpu(), epoch)
```


### E) Close the writer when done
```python
writer.close()
```


## 3) Viewing multiple runs & comparing
Every time you change hyperparameters (batch size, optimizer, LR schedule), change `run_name` so TensorBoard groups the results. Point `--logdir` at `runs/` to compare them side‑by‑side.


## 4) Optional extras
- **Images:**
```python
from torchvision.utils import make_grid
grid = make_grid(xb[:16].cpu(), nrow=8, normalize=True)
writer.add_image('batch_examples', grid, global_step)
```
- **PR/ROC curves:** useful for binary/multi-label; see `add_pr_curve`.
- **Embeddings:** `add_embedding` to visualize feature spaces.
- **Profiler:** `torch.profiler` can export traces that TensorBoard reads (advanced).
